# This notebook will implement Deep learning techniques for a particle-in-halo classification framework. Refer to the original publicly notebook in [ChJazhiel notebook](https://github.com/ChJazhiel/ML_ICF/blob/master/RF_Particles_z23.ipynb)

## For starters, we need to import the libraries and some other utilities 

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

## Import our dataset 

In [ ]:
data_dict = np.load('/home/jazhiel/ML_Notebooks/Cosmology_ML/OUTFILE1M.npz') #np.load('/path/to/nbody/outfile.npz')
test_flags  = data_dict['test_flags'] ## not important
test_hosts  = data_dict['test_hosts'] ### somewhat relevant
test_mass   = data_dict['test_mass'] ## important
test_labels = data_dict['test_labels'] ## important
test_input  = data_dict['test_input'] ## very important
#test_snid   = dict_data['test_snid']
#test_labels = dict_data['test_labels']
print(test_mass) ## Here I want to check how is the halo mass matrix composed of, the -1 means the halo 
#is not in our range of 10^12-10^13 M_sun
print(np.sum(test_labels)) ## Here I want to check how many label "1" do we have

## Data preprocessing

In [ ]:
### here we create or 10 vector dataset, I wonder if by adding some other information the classification will be better
### adding mass is not helpful, the classifier is perfect in that regard
dr1 = pd.DataFrame(test_input[0], columns = ['dr1'])
dr2 = pd.DataFrame(test_input[1], columns = ['dr2'])
dr3 = pd.DataFrame(test_input[2], columns = ['dr3'])
dr4 = pd.DataFrame(test_input[3], columns = ['dr4'])
dr5 = pd.DataFrame(test_input[4], columns = ['dr5'])
dr6 = pd.DataFrame(test_input[5], columns = ['dr6'])
dr7 = pd.DataFrame(test_input[6], columns = ['dr7'])
dr8 = pd.DataFrame(test_input[7], columns = ['dr8'])
dr9 = pd.DataFrame(test_input[8], columns = ['dr9'])
dr10 = pd.DataFrame(test_input[9], columns = ['dr10'])
#mass = pd.DataFrame(test_mass, columns = ['Halo_Mass'])
lbl = pd.DataFrame(test_labels, columns =['labels'])


## Select all features and create dataframe

In [ ]:
df = pd.concat([dr1, dr2, dr3, dr4, dr5, dr6, dr7, dr8, dr9, dr10, lbl], axis=1, ignore_index=False, sort=False)
df

## Here we select a dataframe consisting of evenly separated labels

(I'm not sure if selecting all particles will impact in a different result)


In [ ]:
## Sorting out the labels consisting in label '0' and label '1'
## Then we sample them in order to not selecting them in a specific range or shape

df_0 = df.sort_values('labels').head(710036).sample(289000)
df_1 = df.sort_values('labels').tail(289964).sample(289000) 
df_1.labels.sum()
df_r = pd.concat([df_0, df_1])
#df_scaled = scaler.fit_transform(df_r.drop(['labels'], axis = 1))  
#df_copy1 = pd.DataFrame(df_scaled, columns=['dr1', 'dr2', 'dr3', 'dr4', 'dr5', 'dr6', 'dr7', 'dr8', 'dr9', 'dr10'])

shuffle_df = df_r.sample(frac = 1.0)
#shuffle_df.insert(10, "Model", ['LC','SF']*14300, True)
shuffle_df

## define a size for our traning dataset 

I think that for 500k + particles we can divide into train, test and validation datasets

In [ ]:
# Define a size for your train set 
train_size = int(0.8 * len(df_r))

# Split your dataset 
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

train_set

## Select data, X for attributes, y for labels

In [ ]:
X = shuffle_df.drop(['labels'], axis = 1)
ylabels = shuffle_df.labels
#ymodel = shuffle_df.Model
#ymodel = copy1.Model
X_train = train_set.drop(['labels'], axis = 1)
X_test = test_set.drop(['labels'], axis= 1)
y_trainl = train_set.labels
y_testl = test_set.labels
#y_trainM = train_set.Model
#y_testM = test_set.Model
print(X_train.shape)

## For the Confusion matrix 


In [ ]:
#### THIS CELL IS FOR CALCULATE THE CONFUSION MATRIX ####


import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
    


np.set_printoptions(precision=2)

## Import utilities from scikitlearn, models, evaluation metrics, Random Forest classifier is the model and the evaluation metric is accuracy score

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

## Random forest with sklearn 

In [ ]:
rf=RandomForestClassifier(n_estimators=100, max_depth=8, n_jobs=-1,
                          criterion = 'entropy', class_weight='balanced')

## n_jobs = -1 tells my computer to use all its cores, I have only 2, so it runs on 2 cores
#Train the model using the training sets y_pred=clf.predict(X_test)
rf.fit(X_train,y_trainl)

ypred = rf.predict(X_test)
#####

print('Traning and Testing on raw data, all features \n');
#### Model accuracy
print("Accuracy:", metrics.accuracy_score(y_testl, ypred))

for i, score_forest in enumerate(cross_val_score(rf, X, ylabels, cv = 3)):
    print('Random Forest accuracy for the %d score: %0.2f' % (i, score_forest))
score_forest=cross_val_score(rf, X, ylabels, cv=3)
#score_tree
cv_scores = []
print("Random Forest Accuracy: %0.2f (+/- %0.2f)" % (score_forest.mean(), score_forest.std() * 2 ))
cv_score = score_forest.mean()
cv_scores.append(cv_score)

### We perform a cross validation score to see how accurate our decision tree is by 
### splitting the dataset into 10 folds for training and testing

## Perceptron with Keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

from keras import layers
from keras import models
from keras import optimizers
from keras import losses
from keras import metrics
from matplotlib.ticker import AutoMinorLocator

#para scikit-learn: (samples,features)
#test_input_T = test_input.T
#X_train, X_test, y_train, y_test = train_test_split(test_input_T,test_labels,test_size=0.25,random_state=None)

#ANN
test1_model = models.Sequential()
test1_model.add(layers.Dense(100,activation='relu',input_shape=(10,)))
test1_model.add(layers.Dense(100,activation='relu'))
#test1_model.add(layers.Dense(32,activation='relu'))
#test1_model.add(layers.Dense(10,activation='relu'))
test1_model.add(layers.Dense(1, activation='sigmoid'))

test1_model.compile(optimizer=optimizers.Adam(lr=0.0001),#RMSprop(lr=0.001),
                    loss=losses.binary_crossentropy,
                    metrics=['accuracy'])

test1_model.summary()

test1_model_history = test1_model.fit(X_train,
                                      y_train,
                                      epochs=50,
                                      batch_size=32,
                                      validation_data=(X_test,y_test),
                                      verbose=1)

#plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(11,4.5))    

results_dict = test1_model_history.history
epochs = range(1,len(results_dict['accuracy'])+1)

#accuracy
acc_values = results_dict['accuracy']
val_acc_values = results_dict['val_accuracy']
    
axs[0].plot(epochs,acc_values,color='black',label='train',linewidth=1.5,linestyle='-')
axs[0].plot(epochs,val_acc_values,color='red',label='test',linewidth=1.5,linestyle='-')
axs[0].set_xlabel('epochs')
minorLocatorX = AutoMinorLocator()
axs[0].xaxis.set_minor_locator(minorLocatorX)
axs[0].set_ylabel('accuracy')
axs[0].set_ylim([0.5,1.02])   
minorLocatorY = AutoMinorLocator()
axs[0].yaxis.set_minor_locator(minorLocatorY)
axs[0].tick_params(which='major', length=6)
axs[0].tick_params(which='minor', length=3, color='black')       
axs[0].legend(loc='lower right')
    
#loss
loss_values = results_dict['loss']
val_loss_values = results_dict['val_loss']
       
axs[1].plot(epochs,loss_values,color='black',label='train',linewidth=1.5,linestyle='-')
axs[1].plot(epochs,val_loss_values,color='red',label='test',linewidth=1.5,linestyle='-')
axs[1].set_xlabel('epochs')
minorLocatorX = AutoMinorLocator()
axs[1].xaxis.set_minor_locator(minorLocatorX)   
axs[1].set_ylabel('loss')
minorLocatorY = AutoMinorLocator()
axs[1].yaxis.set_minor_locator(minorLocatorY)
axs[1].tick_params(which='major', length=6)
axs[1].tick_params(which='minor', length=3, color='black')    
axs[1].legend(loc='upper right')

axs[0].set_title('Training and test accuracy after {} epochs: \n {:.3f} (train), {:.3f} (validation)'
                 .format(len(epochs),acc_values[-1],val_acc_values[-1]));

axs[1].set_title('Training and test loss after {} epochs: \n {:.3f} (train), {:.3f} (validation)'
                 .format(len(epochs),loss_values[-1],val_loss_values[-1]));